# Prep for BigRef

Hello. In this notebook I am preparing AnData objects to create a series of references to train scANVI models on. Eventually these references are envisoned as reference datasets unto which we can overlay new experimental data from scRNAseq expeiments involving human Lupus patients' B cells. 

### Data Sources

Sources of these data include

1. Slight-Webb, S. et al. Ancestry-based differences in the immune phenotype are associated with lupus activity. JCI Insight 8, e169584.

2. Perez, R. K. et al. Single-cell RNA-seq reveals cell type–specific molecular and genetic associations to lupus. Science 376, eabf1970 (2022).

In each case I have downloaded and subseted the single cell RNAseq data to include only B cells and Plasmablast cells. 

The Slight-Webb dataset contains ~20K well-annotated B cells from white and black ancestry groups.

The Perez dataset contains ~150K B cells (not as well annotated) from both white and asian ancestry groups

### Analysis Plan

1. Concatenate the AnData objects from the two studies to create "atlas"
2. Use scANVI to "seed label" larger Perez study cells
3. Subset atlas dataset by disease conditions to create
        * Healthy Contorl Matched Atlas
        * SLE disease Atlas
        * Large Atlas (all cells)
5. Use scANVI-trained models for quering

## Setup and Load Data

In [1]:
import scanpy as sc
import os
import scvi
import torch
import anndata

scvi.settings.seed = 1990

/home/Apps/miniconda3/envs/scvi/lib/python3.9/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/Apps/miniconda3/envs/scvi/lib/python3.9/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/home/Apps/miniconda3/envs/scvi/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [87]:
SW_data_dir="/home/Projects/Scharer_sc/scAtlas_ref/data/SlightWebb/"
SW_file="SlightWebb_Bcell_AnnData.h5ad"

SW_adata_path = os.path.join(SW_data_dir, SW_file)

SW_adata = sc.read_h5ad(SW_adata_path)
SW_adata

AnnData object with n_obs × n_vars = 22574 × 27254
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_CITE', 'nFeature_CITE', 'nCount_HTO', 'nFeature_HTO', 'HTO_maxID', 'HTO_secondID', 'HTO_margin', 'HTO_classification', 'HTO_classification.global', 'hash.ID', 'deMULTIplex.calls', 'deMULTIplex.calls.rescued', 'final.HTO.ID', 'run', 'subject_id', 'ancestry', 'classification', 'age', 'percent_mt', 'percent_hemo', 'IGHA1.IGHA2.diff', 'IGHA1.IGHG1.diff', 'IGHA1.IGHG2.diff', 'IGHA1.IGHG3.diff', 'IGHA1.IGHG4.diff', 'IGHA1.IGHGP.diff', 'IGHA1.IGHD.diff', 'IGHA1.IGHE.diff', 'IGHA1.IGHM.diff', 'IGHA2.IGHG1.diff', 'IGHA2.IGHG2.diff', 'IGHA2.IGHG3.diff', 'IGHA2.IGHG4.diff', 'IGHA2.IGHGP.diff', 'IGHA2.IGHD.diff', 'IGHA2.IGHE.diff', 'IGHA2.IGHM.diff', 'IGHG1.IGHG2.diff', 'IGHG1.IGHG3.diff', 'IGHG1.IGHG4.diff', 'IGHG1.IGHGP.diff', 'IGHG1.IGHD.diff', 'IGHG1.IGHE.diff', 'IGHG1.IGHM.diff', 'IGHG2.IGHG3.diff', 'IGHG2.IGHG4.diff', 'IGHG2.IGHGP.diff', 'IGHG2.IGHD.diff', 'IGHG2.IGHE.diff', 'IGHG2

In [19]:
Per_data_dir="/home/Projects/Scharer_sc/scAtlas_ref/data/Perez/"
Per_file="GSE174188_CLUES1_adjusted.h5ad"

Per_adata_path = os.path.join(Per_data_dir, Per_file)

Per_adata = sc.read_h5ad(Per_adata_path)
Per_adata

AnnData object with n_obs × n_vars = 1263676 × 1999
    obs: 'batch_cov', 'ind_cov', 'Processing_Cohort', 'louvain', 'cg_cov', 'ct_cov', 'L3', 'ind_cov_batch_cov', 'Age', 'Sex', 'pop_cov', 'Status', 'SLE_status'
    var: 'gene_ids'
    uns: 'neighbors', 'pca', 'rank_genes_groups', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

I need to do some adjusting to the Perez dataset to get raw counts and just B cells

In [20]:
raw_adata = sc.AnnData(Per_adata.raw.X)
raw_adata.var_names = Per_adata.raw.var_names
raw_adata.obs_names = Per_adata.obs_names

raw_adata.obs = Per_adata.obs.copy()
raw_adata.var = Per_adata.raw.var.copy()

Bmask = (raw_adata.obs['cg_cov']=='B') | (raw_adata.obs['cg_cov']=='PB')

Per_adata2 = raw_adata[Bmask, :]

Per_adata2.layers["counts"] = Per_adata2.X.copy()

Per_adata2.var.rename(columns={'gene_ids':'name'}, inplace=True)
Per_adata2.var.drop('feature_types-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0', axis=1, inplace=True)

Per_adata2

/tmp/ipykernel_1557040/3903527706.py:12: ImplicitModificationWarning: Setting element `.layers['counts']` of view, initializing view as actual.
  Per_adata2.layers["counts"] = Per_adata2.X.copy()


AnnData object with n_obs × n_vars = 152981 × 32738
    obs: 'batch_cov', 'ind_cov', 'Processing_Cohort', 'louvain', 'cg_cov', 'ct_cov', 'L3', 'ind_cov_batch_cov', 'Age', 'Sex', 'pop_cov', 'Status', 'SLE_status'
    var: 'name'
    layers: 'counts'

In [22]:
Per_adata2.var['mt'] = Per_adata2.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(Per_adata2, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

sc.pp.filter_cells(Per_adata2, min_genes=200)
sc.pp.filter_genes(Per_adata2, min_cells=3)

Per_adata2 = Per_adata2[Per_adata2.obs.n_genes_by_counts < 2500, :]

In [23]:
Per_adata2

View of AnnData object with n_obs × n_vars = 152607 × 21429
    obs: 'batch_cov', 'ind_cov', 'Processing_Cohort', 'louvain', 'cg_cov', 'ct_cov', 'L3', 'ind_cov_batch_cov', 'Age', 'Sex', 'pop_cov', 'Status', 'SLE_status', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'name', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    layers: 'counts'

## Align metadata

One important thing I need to do is align the metadata between the two studies. Important attributes include: batch, Coarse Labels, Fine Labels, Sex, Age, Ancestry, Disease State, Disease Status

['batch', 'coarse_lbl', 'fine_lbl','sex','age', 'ancestry', 'disease_state', 'disease_status', 'indiv']

In [88]:
#Creating Disease State and Disease Status variables
import pandas as pd
import numpy as np

SW_adata.obs['disease_state'] = np.where(
    SW_adata.obs['classification'] == 'Control', 
    'Control', 
    np.where(
        SW_adata.obs['classification'].str.contains('SLE'), 
        'Disease', 
        'Unknown'  # This handles any cases that do not match the above conditions
    )
)

In [89]:
#Standardizing names of var columns to above
SW_names = ['run', 'coarse_cell_type', 'fine_cell_type', 'classification', 'subject_id']
names = ['batch', 'coarse_lbl', 'fine_lbl', 'disease_status', 'indiv']

rename_dict = dict(zip(SW_names, names))

existing_columns = [name for name in SW_names if name in SW_adata.obs.columns]
if existing_columns:
    SW_adata.obs.rename(columns=rename_dict, inplace=True)

In [90]:
#Adding sex variable
SW_adata.obs['sex'] = 'Female'

In [91]:
#Dropping other metadata
aligned_obs= ['batch', 'coarse_lbl', 'fine_lbl','sex','age', 'ancestry', 'disease_state', 'disease_status', 'indiv']

columns_to_drop = [col for col in SW_adata.obs.columns if col not in aligned_obs]
SW_adata.obs.drop(columns=columns_to_drop, inplace=True)

In [92]:
SW_adata

AnnData object with n_obs × n_vars = 22574 × 27254
    obs: 'batch', 'indiv', 'ancestry', 'disease_status', 'age', 'coarse_lbl', 'fine_lbl', 'disease_state', 'sex'
    var: 'name'
    uns: 'log1p'
    obsm: 'X_harmony', 'X_harmony_cite', 'X_pca', 'X_pca_cite', 'X_umap_wnn'
    layers: 'counts'

In [76]:
Per_adata2

View of AnnData object with n_obs × n_vars = 152607 × 21429
    obs: 'batch', 'ind_cov', 'Processing_Cohort', 'louvain', 'coarse_lbl', 'fine_lbl', 'L3', 'ind_cov_batch_cov', 'age', 'sex', 'ancestry', 'disease_status', 'disease_state', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'name', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    layers: 'counts'

In [95]:
#Standardizing names of var columns to above
Per_names = ['batch_cov', 'cg_cov', 'ct_cov', 'pop_cov','SLE_status', 'Status', 'Sex', 'Age', 'ind_cov']
names = ['batch', 'coarse_lbl', 'fine_lbl', 'ancestry','disease_state', 'disease_status', 'sex', 'age', 'indiv']

rename_dict = dict(zip(Per_names, names))

existing_columns = [name for name in Per_names if name in Per_adata2.obs.columns]
if existing_columns:
    Per_adata2.obs.rename(columns=rename_dict, inplace=True)

In [96]:
#Dropping other metadata
aligned_obs= ['batch', 'coarse_lbl', 'fine_lbl','sex','age', 'ancestry', 'disease_state', 'disease_status', 'indiv']

columns_to_drop = [col for col in Per_adata2.obs.columns if col not in aligned_obs]
Per_adata2.obs.drop(columns=columns_to_drop, inplace=True)

In [97]:
Per_adata2

AnnData object with n_obs × n_vars = 152607 × 21429
    obs: 'batch', 'indiv', 'coarse_lbl', 'fine_lbl', 'age', 'sex', 'ancestry', 'disease_status', 'disease_state'
    var: 'name', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    layers: 'counts'

## Concatenate datasets

I've had soem internal debates here around how to concatentate these datasets. I suspect that there are differences in the features measured. Perhaps the best thing is to find the HVG in one dataset and set the other to that??

Ultimately I am going to go with the inner join product (overlaps of features seen between the two studies)

In [98]:
import anndata

adata_full = anndata.concat([SW_adata, Per_adata2],merge="same", label="study", keys=["SW", "Per"])

In [99]:
adata_full

AnnData object with n_obs × n_vars = 175181 × 15123
    obs: 'batch', 'indiv', 'ancestry', 'disease_status', 'age', 'coarse_lbl', 'fine_lbl', 'disease_state', 'sex', 'study'
    layers: 'counts'

In [123]:
adata_full.obs['disease_state'].value_counts()

disease_state
SLE        106005
Healthy     69176
Name: count, dtype: int64

In [122]:
#Aligning levels
adata_full.obs['ancestry'].replace('AA', 'African American', inplace=True)
adata_full.obs['ancestry'].replace('EA', 'European', inplace=True)

adata_full.obs['disease_state'].replace('Disease', 'SLE', inplace=True)
adata_full.obs['disease_state'].replace('Control', 'Healthy', inplace=True)

adata_full.obs['coarse_lbl'].replace('B cells', 'B', inplace=True)
adata_full.obs['coarse_lbl'].replace('Plasmablasts', 'PB', inplace=True)

In [115]:
adata_full.obs['labels'] = adata_full.obs['fine_lbl']
adata_full.obs['labels'] = adata_full.obs['labels'].cat.add_categories(['Unknown'])
adata_full.obs.loc[adata_full.obs['study'] == 'Per', 'labels'] = 'Unknown'
adata_full.obs['labels'] = adata_full.obs['labels'].cat.remove_unused_categories()

In [119]:
adata_full

AnnData object with n_obs × n_vars = 175181 × 15123
    obs: 'batch', 'indiv', 'ancestry', 'disease_status', 'age', 'coarse_lbl', 'fine_lbl', 'disease_state', 'sex', 'study', 'labels'
    layers: 'counts'

In [124]:
#Subset by Disease & Healthy
adata_SLE = adata_full[adata_full.obs['disease_state'] == 'SLE'].copy()
adata_HC = adata_full[adata_full.obs['disease_state'] == 'Healthy'].copy()

## Saving AnData Object

In [125]:
adata_full.write("BcellRefAtlas_SLEHC.h5ad")
adata_SLE.write("BcellRefAtlas_SLE.h5ad")
adata_HC.write("BcellRefAtlas_HC.h5ad")